In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedShuffleSplit, KFold
from imblearn.over_sampling import SMOTE

In [3]:
df = pd.read_csv("Creditcard_data.csv")

In [4]:
# Oversampling minority class using SMOTE
X1 = df.drop('Class', axis=1)
y1 = df['Class']
smote = SMOTE(random_state=40)
X_smote, y_smote = smote.fit_resample(X1, y1)
X = X_smote.values
y = y_smote.values

In [5]:
# Define the five different machine learning models
M1 = GaussianNB()
M2 = LogisticRegression(random_state=40, solver='lbfgs', max_iter=100)
M3 = KNeighborsClassifier()
M4 = SVC(random_state=40)
M5 = DecisionTreeClassifier(random_state=40)
# Didnt chose Random forest as it was giving 1 accuracy with all sampling techniques and it was looking Invalid.

In [6]:
# Define the five different sampling techniques
Sampling1 = 'Random Sampling'
Sampling2 = 'Stratified Sampling'
Sampling3 = 'Cluster Sampling'
Sampling4 = 'OverSampling'
Sampling5 = 'Undersampling'

In [7]:
# Define a function to apply the different sampling techniques to the different machine learning models
def apply_sampling(model, sampling_technique, X, y):
    if sampling_technique == Sampling1:
        train_index, test_index = train_test_split(range(X.shape[0]), test_size=0.60, random_state=101)
        X_sampled, y_sampled = X[train_index], y[train_index]

    elif sampling_technique == Sampling2:
        sss = StratifiedShuffleSplit(n_splits=1, test_size=0.60, random_state=101)
        train_index, test_index = next(sss.split(X, y))
        X_sampled, y_sampled = X[train_index], y[train_index]

    elif sampling_technique == Sampling3:
        kf = KFold(n_splits=5, shuffle=True, random_state=101)
        cluster_indices = []
        for _, test_index in kf.split(X):
            cluster_indices.append(test_index)
        test_index = np.concatenate(cluster_indices)
        X_sampled, y_sampled = X[test_index], y[test_index]

    elif sampling_technique == Sampling4:
        ros = RandomOverSampler(random_state=101)
        X_sampled, y_sampled = ros.fit_resample(X, y)

    elif sampling_technique == Sampling5:
        rus = RandomUnderSampler(random_state=101)
        X_sampled, y_sampled = rus.fit_resample(X, y)

    # Split the sampled data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_sampled, y_sampled, test_size=0.3, random_state=40)

    # Fit the machine learning model on the training data and evaluate it on the testing data
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)

    return accuracy

In [8]:
# Define the list of models and sampling techniques to evaluate
models = [M1, M2, M3, M4, M5]
sampling_techniques = [Sampling1, Sampling2, Sampling3, Sampling4, Sampling5]

In [9]:
# Evaluate each model with each sampling technique and store the results in a matrix
results = np.zeros((len(models), len(sampling_techniques)))
for i, model in enumerate(models):
    for j, technique in enumerate(sampling_techniques):
            accuracy = apply_sampling(model, technique, X, y)
            results[i, j] = accuracy

# Convert the results matrix to a pandas DataFrame with row and column names
results_df = pd.DataFrame(results, index=['Naive Bayes', 'Logistic Regression', 'K-Nearest Neighbours', 'Support Vector Machine', 'Decision Tree'],
                                     columns=[Sampling1, Sampling2, Sampling3, Sampling4, Sampling5])

# Print the results DataFrame
print('The Resultant Matrix is:\n')
display(results_df)
print('Maximum Accuracy achieved is ',round(results_df.values.max(), 6))

c:\Users\navya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\navya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

The Resultant Matrix is:



,Random Sampling,Stratified Sampling,Cluster Sampling,OverSampling,Undersampling
Naive Bayes,0.765027,0.836066,0.873362,0.812227,0.882096
Logistic Regression,0.907104,0.896175,0.927948,0.910480,0.893013
K-Nearest Neighbours,0.775956,0.765027,0.842795,0.812227,0.814410
Support Vector Machine,0.693989,0.704918,0.676856,0.679039,0.661572
Decision Tree,0.989071,0.950820,0.984716,0.984716,0.975983


Maximum Accuracy achieved is  0.989071
